## <font color=009DD2> <span style="background-color: #A9C4C0"> Imports

In [1]:
import sys,json
import glob, os
sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from seed_to_voxels_inprogress_nk import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

## <font color=009DD2> <span style="background-color: #A9C4C0"> Initialization

In [2]:
with open('../config/config_NK.json') as config_file:
    config = json.load(config_file) # load config file

seed_names=['icap1'] # Define seed names (here, icap numbers)
target_name=['brain'] # Define structure to use as target (brain or spinal cord)
seed2voxels=Seed2voxels(config,seed_names,target_name) # initialize the function
ktot = 9 # Total number of iCAPs to use

## <font color=009DD2> <span style="background-color: #A9C4C0"> Data extraction
### <font color=009DD2> I Select files </font>

In [3]:
data_target=[];ts_seed_txt=[];ts_target_txt=[]
for seed_name in seed_names:
    for subject_name in config['list_subjects']:
        data_target.append(config['main_dir'] + '1_nilearn-based/3_func_smoothed/sub-'+ subject_name + '/TA_results/MNI/TotalActivation/Activity_inducing.nii')
        ts_seed_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + ktot '/'+seed_name+'/timeseries/sub-' + subject_name + '_' + seed_name + '_timeseries_ai')
        ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/sub-' + subject_name + '_' + target_name[0] + '_all_timeseries_ai')
        
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + ktot ''+seed_name+'/'+target_name[0]+'_fc_maps/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + ktot '/'+seed_name+'/'+target_name[0]+'_fc_maps/')

mask_target=config['main_dir'] + config['masks'][target_name[0].split('_')[0]]
 
# Create directories if needed
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/'):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all')
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/') # folder to store timeseries extraction


### <font color=009DD2> II. Time series extraction </font>

In [16]:
seed_timeseries=seed2voxels.extract_data(signal='ai',timeseries_txt=ts_seed_txt,
                                         run="load",n_jobs=8) # Load iCAP timecourses as seed

In [19]:
target_timeseries=seed2voxels.extract_data(signal='ai',img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                run="load",n_jobs=8) # Load voxelwise brain AI as target

## <font color=009DD2> <span style="background-color: #A9C4C0"> Correlation analysis

In [22]:
output_img={};
for seed_name in seed_names:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) + '/'+seed_name+'/'+target_name[0]+'_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_' + seed_name + '_zcorr.nii'

seed2voxels.correlation_maps(seed_timeseries,target_timeseries,mask=mask_target,output_img=output_img[seed_name],Fisher=True,n_jobs=8)



IndexError: tuple index out of range

## <font color=009DD2> <span style="background-color: #A9C4C0"> Smoothing

In [28]:
from nilearn import image
img_smoothed=image.smooth_img(output_img[seed_name],[6,6,6])
img_smoothed.to_filename(output_img[seed_name].split('.')[0] + '_s.nii' )